In [ ]:
# Importing necessary libraries
import pandas as pd
import sqlite3

# Load the comments.csv and videos-stats.csv files
comments_df = pd.read_csv('Projects/comments.csv')
video_stats_df = pd.read_csv('Projects/videos-stats.csv')

# Display the first few rows of each dataframe to confirm successful loading
comments_df.head(), video_stats_df.head()

In [ ]:
import sqlite3

# Create a new SQLite database in memory
conn = sqlite3.connect(':memory:')

# Create tables for videos_stats and comments
comments_df.to_sql('comments', conn, index=False)
video_stats_df.to_sql('video_stats', conn, index=False)

In [ ]:
import pandas as pd

# Load the CSV files into DataFrames
videos_stats = pd.read_csv('Projects/videos-stats.csv')
comments = pd.read_csv('Projects/comments.csv')

# Show the first few rows of each DataFrame to confirm they loaded correctly
videos_stats.head(), comments.head()

In [ ]:
-- Question: Which videos have the highest and lowest like-to-dislike ratios?

-- Highest like-to-dislike ratio
SELECT video_id, title, (likes / dislikes) AS like_to_dislike_ratio
FROM video_stats
ORDER BY like_to_dislike_ratio DESC
LIMIT 1;

-- Lowest like-to-dislike ratio
SELECT video_id, title, (likes / dislikes) AS like_to_dislike_ratio
FROM video_stats
WHERE dislikes > 0
ORDER BY like_to_dislike_ratio ASC
LIMIT 1;

-- Question: What is the average number of comments per video for each category?

SELECT category, AVG(comments) AS avg_comments_per_video
FROM video_stats
GROUP BY category
ORDER BY avg_comments_per_video DESC;

-- Question: Which day of the week receives the most comments?

-- Calculate the day of the week (0 = Sunday, 1 = Monday, ..., 6 = Saturday) and count comments for each day.
-- Order the results by comment count in descending order and limit to the top result.
SELECT strftime('%w', datetime(published_at, 'unixepoch')) AS day_of_week, COUNT(*) AS comment_count
FROM comments
GROUP BY day_of_week
ORDER BY comment_count DESC
LIMIT 1;

-- Question: What are the top 3 most frequent words used in comments?

-- Split comments into words, count their occurrences, and return the top 3 words.
WITH words AS (
    SELECT word, COUNT(*) AS word_count
    FROM (
        SELECT DISTINCT regexp_split_to_table(lower(comment_text), E'\\\\s+') AS word
        FROM comments
    ) AS words
    GROUP BY word
)
SELECT word, word_count
FROM words
ORDER BY word_count DESC
LIMIT 3;

-- Question: How many videos have more likes than the average number of likes across all videos?

-- Query to find the number of videos with more likes than the average
SELECT COUNT(*) AS videos_with_more_likes
FROM video_stats
WHERE likes > (SELECT AVG(likes) FROM video_stats);

-- Question: What percentage of total views does each category represent?

-- Query to calculate the percentage of total views for each category
SELECT category, SUM(views) AS total_views, (SUM(views) * 100.0 / (SELECT SUM(views) FROM video_stats)) AS percentage_of_total_views
FROM video_stats
GROUP BY category
ORDER BY percentage_of_total_views DESC;

-- Question: Are there any videos that appear in the top 5 for both most likes and most comments?

-- Query to find videos in the top 5 for both most likes and most comments
WITH top_likes AS (
    SELECT video_id
    FROM video_stats
    ORDER BY likes DESC
    LIMIT 5
), top_comments AS (
    SELECT video_id
    FROM video_stats
    ORDER BY comments DESC
    LIMIT 5
)
SELECT v.title, v.likes, v.comments
FROM video_stats v
WHERE v.video_id IN (SELECT video_id FROM top_likes)
    AND v.video_id IN (SELECT video_id FROM top_comments);

-- Question: What is the average number of views for videos that have above-average likes?

-- Query to calculate the average number of views for videos with above-average likes
WITH avg_likes AS (
    SELECT AVG(likes) AS avg_likes
    FROM video_stats
)
SELECT AVG(views) AS avg_views_above_avg_likes
FROM video_stats
WHERE likes > (SELECT avg_likes FROM avg_likes);


In [ ]:
-- Query to find videos with sentiment scores higher than the average sentiment score
WITH avg_sentiment AS (
    SELECT AVG(sentiment_score) AS avg_sentiment_score
    FROM video_stats
)
SELECT video_id, title, sentiment_score
FROM video_stats
WHERE sentiment_score > (SELECT avg_sentiment_score FROM avg_sentiment)
ORDER BY sentiment_score DESC;


In [ ]:
-- Query to calculate the correlation between views and comments
SELECT CORR(views, comments) AS correlation
FROM video_stats;
